In [3]:
# question_templates={
#     'missing_step' : 'Qual è lo step mancante nella seguente lista di step per la guida: {title}. Steps: {list_of_steps}',

#     'next_step' : 'Sapendo che il titolo della guida è {step}.Qual è il prossimo step in questa lista: {list_of_steps}',

#     'open_question' : 'Write the steps for this guide: {title}',

#     'rearrange' : 'Rearrange the steps for {title}: {list_of_steps}'
# }

# Questions
- remove a single step but print in the question all steps: step1 step2 ... step4 -> correct answer: step3
- predict the next step (minimum 4 steps before): step1 2 3 4 ... -> correct answer: step5
- Sequence Rearrangement. Provide the steps in a jumbled order and ask the user to arrange them in the correct   sequence.
Example:
Question: Rearrange the steps for “How to Tie a Shoelace”: Step 3, Step 1, Step 2, Step 4.
Correct Answer: Step 1, Step 2, Step 3, Step 4.
- open question write the steps for this guide
- difficulty: take the option step between the same guide (maximum difficulty) or from a different guide (easy)

In [4]:
import requests
from bs4 import BeautifulSoup,NavigableString
import random
from tqdm import tqdm
from multiprocessing import Pool, cpu_count


In [5]:
url='https://www.wikihow.com/Category:Gardening'
def get_all_pages(url,releted=True):
    base_url='https://www.wikihow.com'
    response = requests.get(url)
    pages=set()
    categories=set()
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        cat_sections=soup.find_all('div',class_='cat_section')
        for cat_section in cat_sections:
            if cat_section.get('id')=='cat_featured':
                #print('Feautured')
                articles=cat_section.find_all('div',class_='responsive_thumb')
                for article in articles:
                    url_a=article.find('a').get('href')
                    pages.add(url_a)
            if cat_section.get('id')=='cat_all':
                #print('ALL')
                articles=cat_section.find_all('div',class_='responsive_thumb')
                for article in articles:
                    url_a=article.find('a').get('href')
                    pages.add(url_a)
            if cat_section.get('id')=='cat_sub_categories' and releted:
                #print('SUBCAT')
                uls=cat_section.find_all('ul')
                for ul in uls:
                    for li in ul.find_all('li'):
                        #print(li)
                        for a in li.find_all('a',class_='cat_link'):
                            categories.add(base_url+a.get('href'))
        return pages,categories

In [6]:
urls,categories=get_all_pages(url,True)


In [7]:
urls2= set()
for category in categories:
    u,c = get_all_pages(category,False)
    urls2.update(u)
    if c:
        print(c)

In [8]:
urls

{'https://www.wikihow.com/Adjust-Hunter-Sprinklers',
 'https://www.wikihow.com/Apply-Liquid-Fertilizer',
 'https://www.wikihow.com/Apply-Mulch',
 'https://www.wikihow.com/Arrange-the-Inside-of-a-Greenhouse',
 'https://www.wikihow.com/Build-a-Lean-to-Shed',
 'https://www.wikihow.com/Build-a-Pergola',
 'https://www.wikihow.com/Clean-Garden-Stones',
 'https://www.wikihow.com/Clean-a-Slate-Hearth',
 'https://www.wikihow.com/Cut-Back-Irises-in-the-Fall',
 'https://www.wikihow.com/Cut-Back-Penstemon',
 'https://www.wikihow.com/Deal-with-Exposed-Tree-Roots',
 'https://www.wikihow.com/Determine-How-Much-Water-Plants-Need',
 'https://www.wikihow.com/Garden',
 'https://www.wikihow.com/Get-Rid-of-Morning-Glories',
 'https://www.wikihow.com/Grow-Bloodworms',
 'https://www.wikihow.com/Grow-Daikon',
 'https://www.wikihow.com/Grow-Pumpkins-Indoors',
 'https://www.wikihow.com/Grow-Sweet-Potato-Vine-Houseplant',
 'https://www.wikihow.com/Grow-Taro',
 'https://www.wikihow.com/Grow-Zucchini-in-Pots',
 'h

In [9]:
#url = 'https://www.wikihow.com/Prune-a-Tree'
def get_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        return {'title': url.split('/')[-1], 'soup':soup}
    else:
        print("Failed to retrieve the page.")

In [10]:
urls.update(urls2)

In [11]:
soups=[]
for url in tqdm(list(urls)):
    soup = get_page(url)
    soups.append(soup)

100%|██████████| 997/997 [03:45<00:00,  4.42it/s]


In [306]:
def process_soup(title,soup):
    title=title
    if title in ['Prune-Nandinas','Kill-Poison-Oak','What-Plant-Am-I','Meaning-of-the-Carnation','Grow-Mint-from-Cuttings']:
        return None
    #sections = soup.find_all('div',class_=lambda x: x and x.startswith('section steps'))
    sections = soup.find_all('div', class_=lambda x: x and (x in ['section steps steps_first sticky hide_step_numbers','section steps sticky hide_step_numbers','section steps before_steps_section sticky', 'section steps steps_first sticky', 'section steps sticky']))# == 'section steps steps_first sticky' or x == 'section steps sticky'))
    
    sections_dir=[]
    if len(sections)==0:
        print('No Section',title)
        return None
    for section in sections:
        h3_tag = section.find_all('h3')
        type = section.find_all('div',class_='altblock')
        section_type = 'No type'
        if type:
            section_type = type[0].text.strip()
        #print(type[0].text)
        if h3_tag:
            section_title = h3_tag[0].text.strip()
            #print(title,' -> ',section_title)
            #sections_dir.append(section_title)
        else:
            h2_tag = section.find_all('h2')
            if h2_tag:
                section_title = h2_tag[0].text.strip()
                #sections_dir.append(section_title)
        
        ols=section.find_all('ol',class_=lambda x: x and x.startswith('steps'))
        if len(ols)>1:
            print('Attention more than one ol',title)

            #return
        #part=0
        page=[]
        
        divs=[]
        subsections=[]
        for ol in ols:
            #section=[]
            #part+=1
            step_num=0
            subsection=[]
            for li in ol.find_all('li', id=lambda x: x and x.startswith('step-id')):
                step={}
                step_num+=1
                img_tag = li.find('img')
                if img_tag and img_tag.get('data-src'):
                    step_image = img_tag['data-src']
                    step['image'] = step_image
                else:
                    # Handle missing image or data-src attribute
                    step['image'] = None
                step_div=li.find('div', class_='step')
                for element in step_div.children:
                    if isinstance(element, NavigableString) and element.strip():
                        #non_tag_text += element.strip() + " "
                        text = step.get('text','')
                        text += element.strip() + " "
                        step['text'] = text
                    # Check if element has the class 'whb'
                    elif element.name and 'whb' in element.get("class", []):
                        step_title = step.get('title','')
                        step_title += element.get_text(strip=True)
                        step['title'] = step_title
                    elif element.name == 'a' and 'external' in element.get("class", []):
                        #non_tag_text += element.text + " "
                        text = step.get('text','')
                        text += element.text + " "
                        step['text'] = text
                    elif element.name == 'span' and 'frac' in element.get("class", []):
                        #non_tag_text += element.text + " "
                        text = step.get('text','')
                        fraction = element.text.replace('⁄','/')
                        text += fraction + " "
                        step['text'] = text
                    elif element.name == 'span' and 'mwe-math-element' in element.get("class", []):
                        #non_tag_text += element.text + " "
                        text = step.get('text','')
                        fraction = element.text.replace('⁄','/')
                        text += fraction + " "
                        step['text'] = text
                    elif element.name == 'a' and (not element.get("class") or 'intra-article' in element.get("class")):
                        #non_tag_text += element.text + " "
                        found=False
                        for child in element.children:
                            if child.name and 'whb' in child.get("class", []):
                                step_title = step.get('title','')
                                step_title += child.get_text(strip=True)
                                step['title'] = step_title 
                                found=True
                        if not found:
                            text = step.get('text','')
                            text += element.text + " "
                            step['text'] = text
                    elif element.name == 'i' and not element.get("class"):
                        #non_tag_text += element.text + " " 
                        text = step.get('text','')
                        text += element.text + " "
                        step['text'] = text
                    elif element.name == 'b' and not element.get("class"):
                        #non_tag_text += element.text + " "
                        found=False
                        for child in element.children:
                            if child.name and 'whb' in child.get("class", []):
                                step_title = step.get('title','')
                                step_title += child.get_text(strip=True)
                                step['title'] = step_title 
                                found=True
                        if not found:
                            text = step.get('text','')
                            text += element.text + " "
                            step['text'] = text
                    elif element.name == 'div':
                        #print(f'Div Found in part {section_type} step: {step_num} | Article: {title}')
                        divs.append(element)
                    elif element.name == 'ul':
                        ul_elem=[]
                        for li in element.find_all('li'):
                            plain_text = ''.join(text for text in li.contents if isinstance(text, str)).strip()
                            ul_elem.append(plain_text)
                        step_list = ''
                        for elem in ul_elem:
                            step_list += '\n- ' + elem
                        text = step.get('text','')
                        text += step_list + " "
                        step['text'] = text
                        #step['ul_lists']= step_list
                    # Check if element is another tag
                    elif element.name in ['script', 'sup','br','p','span','table']: #table could be processed
                        pass
                    elif element.name:
                        print(f"Other tag ({element.name}) found in part {section_type} step: {step_num} | Article: {title} |: {element}")
                    else:
                        pass
                if not step.get('title') or len(step['title'])<5:
                    print('Problem: ',title,'in the step',step.get('title'),f'found in part {section_type} step: {step_num}')
                subsection.append(step) 
            subsections.append(subsection)
        sections_dir.append({'section_title' : section_title,'section_type' : section_type, 'subsections':subsections})
    return {'title':title,'sections' : sections_dir}#, divs

In [366]:
pages={}
for soup in tqdm(soups):
    processed_soup=process_soup(soup['title'],soup['soup'])
    if processed_soup:
        page=Page(processed_soup)
        if  page.sections[0].type.startswith('1'):
            adjust_page(page)
    #Page(pages['Plant-an-Avocado-Tree'])
        pages[soup['title']] = page

 20%|██        | 201/997 [00:01<00:05, 143.83it/s]

Attention more than one ol Plant-an-Avocado-Tree


 27%|██▋       | 265/997 [00:01<00:04, 151.30it/s]

Attention more than one ol Plant-a-Mango-Seed


 35%|███▌      | 353/997 [00:02<00:03, 166.01it/s]

...

 60%|█████▉    | 597/997 [00:03<00:02, 199.65it/s]

Attention more than one ol Grow-Edible-Mushrooms


 66%|██████▌   | 659/997 [00:03<00:01, 188.62it/s]

Attention more than one ol Sharpen-Lawn-Mower-Blades


100%|██████████| 997/997 [00:05<00:00, 168.36it/s]


In [353]:
def all_one(page):    
    for section in page.sections:
        if len(section.steps)>1:
            return False
    return True

In [365]:
def adjust_page(page):
    steps=[]
    for num,section in enumerate(page.sections,1):
        new_step=Step({'title' : section.title,'text' : section.steps[0].title+' '+section.steps[0].text,'image':section.steps[0].image},index=num)
        #print(new_step)
        steps.append(new_step)
    page.sections[0].steps = steps
    page.sections[0].type='No type'
    page.sections[0].title='Steps'
    #page.sections[0].steps = steps
    page.sections = [page.sections[0]]

In [381]:
c=0
types=set()
for page in pages:
    #if pages[page].sections[0].type == 'No section':
    types.add( pages[page].sections[0].type)
    #if  not pages[page].sections[0].type:#.startswith('No'):
    if  pages[page].sections[0].type.startswith('M'):
        print(pages[page])
        c+=1
        #adjust_page(pages[page])
        #print(pages[page])
print(c,'/',len(pages))

Page Title: Plant-Lettuce
URL: https://www.wikihow.com/Plant-Lettuce
Sections:
- (Method 1) Growing Head Lettuce  with 8 steps.
   -- Step 1: Choose a head lettuce variety for indoor planting. - Head lettuces take longer to mature. If you start ...
   -- Step 2: Prepare seed trays. - You can start your lettuce seeds in store-bought s...
   -- Step 3: Sow the seeds 4-6 weeks before the last spring frost. - This will give them time to germinate and sprout b...
   -- Step 4: Give the seeds plenty of sunlight and water. - Place the tray in a sunny window and keep the grow...
   -- Step 5: Transplant into the garden. - The earliest you can transplant your seedlings is ...
   -- Step 6: Fertilize the lettuce three weeks after transplanting. - Use alfalfa meal or slow release fertilizer that i...
   -- Step 7: Cut mature leaves. - When the leaves look mature enough to eat, resembl...
   -- Step 8: Store harvested lettuce in the refrigerator. - If you don't eat your lettuce right away, you can

In [368]:
types

{'', 'Method 1', 'No type', 'Part 1'}

In [451]:
page_t=list(pages.keys() )[random.randint(0,len(pages))]
print(pages[page_t])#['sections'][0]['steps'][0]

Page Title: Deter-Possums-from-Your-Garden
URL: https://www.wikihow.com/Deter-Possums-from-Your-Garden
Sections:
- (Method 1) Blocking Access  with 4 steps.
   -- Step 1: Know what to protect. - Possums seem to be fond of new growth on deciduous...
   -- Step 2: Protect individual fruits and vegetables. - For example, there are cloth bags that are made to...
   -- Step 3: Enclose garden beds. - Instead of keeping possums out of your yard, you c...
   -- Step 4: Block all entry points. - Possums may make their way into your eaves or othe...

- (Method 2) Deterring Climbing  with 4 steps.
   -- Step 1: Make it hard for possums to climb structures or plants. - If the possum is using a structure to get to the p...
   -- Step 2: Trim trees that are being eaten. - If a possum is eating the new growth off a tree, y...
   -- Step 3: Put netting on fruit bushes and trees. - In particular, you should place netting over these...
   -- Step 4: Place a greased PVC pipe on top of your fence. - This 

In [452]:
class Page:
    def __init__(self, data):
        self.title = data['title']
        self.sections = [Section(section) for section in data['sections']]

    def display_sections(self):
        return [section.display() for section in self.sections]
    
    def __str__(self):
        output = f'Page Title: {self.title}\nURL: https://www.wikihow.com/{self.title.replace(" ","-")}\nSections:\n'
        for section in self.sections:
            output += str(section) + '\n'
        return output

class Section:
    def __init__(self, section_data):
        self.title = section_data['section_title']
        self.type = section_data['section_type']
        self.steps = []
        for sub_i ,subsection in enumerate(section_data['subsections']):
            #self.subsection.append([])
            for i,step_data in enumerate(subsection,1):
                self.steps.append(Step(step_data, i))

    def display(self):
        return f'({self.type}) {self.title} with {len(self.steps)} steps.'
    
    def __str__(self):
        output = f'- ({self.type}) {self.title}  with {len(self.steps)} steps.\n'
        for step in self.steps:
            output += str(step) + '\n'
        return output

class Step:
    def __init__(self, step_data, index):
        self.index = index
        self.image = step_data['image']
        self.title = step_data.get('title', '')  # Use get() to provide a default value
        self.text = step_data.get('text', '')   # Use get() to provide a default value

    def display(self):
        return f'       Step {self.index}: {self.title}\n            {self.text[:50]}...'
    
    def __str__(self):
        return f'   -- Step {self.index}: {self.title} - {self.text[:50]}...'

In [468]:
pages['Germinate-Cilantro-Seeds'].sections[0].steps[0].title

'Fill your container, pot, etc.'

In [ ]:
question_templates = {
    'missing_step': 'Steps: {list_of_steps}. What is the missing step in this sequence for "{section_title}" in the page titled "{page_title}"?', # No option # Option with LLM
    'next_step': 'Steps: {list_of_steps}. For the section "{section_title}" in the page titled "{page_title}", what comes next in this sequence?', # No option # Option with LLM
    'open_question': 'Write down all the steps for the section "{section_title}" in the page titled "{page_title}".',
    'rearrange': 'Steps: {list_of_steps}. Put these steps in the correct order for the section "{section_title}" in the page titled "{page_title}".' # No LLM
}

def create_question(page, question_type):
    question = question_templates[question_type]
    list_of_steps=[]
    if question_type == 'missing_step':
        missing_step_num = random.randint(4,len(page['steps'])-1)
        list_of_steps = page['steps'][:]
        #correct_answer = page['steps'][missing_step_num]
        correct_answer = list_of_steps[missing_step_num]
        list_of_steps[missing_step_num] = "......"
    elif question_type == 'next_step':
        correct_answer = page['steps'][-1]
        list_of_steps = page['steps'].remove(-1)
    elif question_type == 'rearrange':
        correct_answer = page['steps']
        list_of_steps = random.shuffle(page['steps'])
    question=question.format(title=page['title'],list_of_steps=list_of_steps)
    options=[] # created using llm
    return {'question' : question, 'options' : options,'correct_answer' : correct_answer}


In [475]:
def create_question(page, question_type):
    question = question_templates[question_type]
    list_of_steps = []
    
    if question_type == 'missing_step':
        missing_step_num = random.randint(4, len(page['steps']) - 2)
        list_of_steps = page['steps'][:]
        correct_answer = list_of_steps[missing_step_num]
        list_of_steps[missing_step_num] = "............................."
    elif question_type == 'next_step':
        correct_answer = page['steps'][-1]
        list_of_steps = page['steps'][:-1]  # Remove the last step without modifying the original list
    elif question_type == 'rearrange':
        correct_answer = page['steps']
        list_of_steps = page['steps'][:]
        random.shuffle(list_of_steps)

    # Join the steps with newline characters
    formatted_steps = "\n".join(list_of_steps)
    question = question.format(title=page['title'], list_of_steps=formatted_steps)
    
    options = []  # Placeholder for options created using LLM
    return {'question': question, 'options': options, 'correct_answer': correct_answer}


In [233]:
question_templates['missing_step'].format(title='Stocazzo',list_of_steps=[])

'What is the missing step in the following sequence for "Stocazzo"?\n Steps: []'

In [756]:
question=create_question(page,'missing_step')
print(question['question'])
print('\n',question['correct_answer'])

What is the missing step in the following sequence for "Get-Rid-of-Snails"?
 Steps: Pick snails off by hand.
Usecopper tapeor wire to discourage snail activity.
Make a beer trap to capture the snails.
Spray the snails with vinegar.
Use a commercial snail bait.
Get a pet chicken that will eat the snails.
.............................
Call a professional if all else fails.
Water your soil in the morning to reduce egg laying.
Remove moist, decaying debris from your yard and garden.
Put up an abrasive barrier to deter snails.
Sprinkle used coffee grounds around your garden.
Add plants that naturally deter snails.
Slime trails
Snail eggs
Holes in leaves
Damaged seedlings
Chewed bark and flower buds
Quarantine plants for 2 weeks before putting them into your aquarium.
Treat plants in a bleach solution before adding them to your tank.
Remove snails by hand by cleaning the aquarium.
Introduce fish that eat snails.
Put a snail trap in your tank to catch and remove the snails.
Add fish-friendly 